In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [2]:
df = pd.read_excel('../database/export_binance/df_new.xlsx')
df

/var/folders/yl/s1pq6fh930j3frb4n30yfhv00000gn/T/ipykernel_46706/2301737341.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel('../database/export_binance/df_new.xlsx')


,Unnamed: 0,User_ID,UTC_Time,Account,Operation,Coin,Change,Remark
0,1,1,2021-05-31 09:00:00,Spot,Deposit,SOL,25.000000,NaN
1,1,1,2021-05-31 09:00:00,Spot,Deposit,BTC,1.000000,NaN
2,1,1,2021-05-31 09:00:00,Spot,Deposit,BNB,35.000000,NaN
3,1,1,2021-05-31 09:00:00,Spot,Deposit,BUSD,50000.000000,NaN
4,1,1,2021-05-31 09:00:00,Spot,Deposit,USDT,50000.000000,NaN
...,...,...,...,...,...,...,...,...
9701,3070,15578312,2022-04-04 12:07:32,USDT-Futures,transfer_in,USDT,113.235373,NaN
9702,3110,15578312,2022-05-01 12:44:01,USDT-Futures,transfer_in,BUSD,221.751928,NaN
9703,3111,15578312,2022-05-01 12:44:01,Spot,transfer_out,BUSD,-221.751928,NaN
9704,3185,15578312,2022-05-11 22:11:08,Spot,transfer_out,USDT,-148.830000,NaN


In [10]:
overview = df.groupby(['Account', 'Coin']).agg({'Change': 'sum'})
overview

Change
Account      Coin               
Card         BNB    1.199195e-01
             EUR    7.270000e+00
Spot         AAVE   1.100000e+00
             ADA    1.306000e+02
             ATOM   3.533031e+01
             AUDIO  2.000000e+02
             AVAX   5.972000e+01
             BETA   0.000000e+00
             BNB    9.426420e+00
             BTC    1.372650e+00
             BUSD   2.140355e+04
             CAKE   4.546600e+02
             CELR   0.000000e+00
             DOT    0.000000e+00
             EDG    1.136780e+02
             EOS    1.421085e-14
             ETH    2.299233e+00
             EUR    6.820933e-14
             FTM    1.110000e+03
             FTT    1.541519e-15
             LINK   8.633900e+01
             LTC    0.000000e+00
             LUNA   3.351500e+02
             MANA   1.100000e+03
             MATIC  2.713000e+02
             NEAR   0.000000e+00
             QUICK  3.000000e+00
             RUNE   0.000000e+00
             SAND   1.136868e-13
             SOL    2.481000e+01
             SUSHI -2.842171e-14
             UNI    3.552714e-15
             USDC   4.947000e-03
             USDT   4.416449e+03
             WETH   0.000000e+00
             XTZ    0.000000e+00
USDT-Futures BNB    1.000000e-07
             BUSD   0.000000e+00
             USDT   1.501694e+03

In [13]:
overview_pivot = df.pivot_table(index='Coin', columns='Account', values='Change', aggfunc=np.sum)
overview_pivot

Account,Card,Spot,USDT-Futures
Coin,,,
AAVE,NaN,1.100000e+00,NaN
ADA,NaN,1.306000e+02,NaN
ATOM,NaN,3.533031e+01,NaN
AUDIO,NaN,2.000000e+02,NaN
AVAX,NaN,5.972000e+01,NaN
BETA,NaN,0.000000e+00,NaN
BNB,0.119919,9.426420e+00,1.000000e-07
BTC,NaN,1.372650e+00,NaN
BUSD,NaN,2.140355e+04,0.000000e+00
